In [1]:
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，但是对于某些交易判断还是有问题，导致结果计算不完全正确
'''

'\ndata_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁\n添加了pnl的计算，但是对于某些交易判断还是有问题，导致结果计算不完全正确\n'

In [2]:
def calculate_pnl(btc_record_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate PnL for BTC transactions based on BTC_record_df.

    :param btc_record_df: DataFrame containing BTC transaction records.
    :return: DataFrame with calculated PnL and summary of Realised and Unrealised PnL.
    """
    
    # Define current price as the price of the last row
    current_price = btc_record_df.iloc[-1]['average_price']
    
    results = []
    total_pnl = 0

    for _, row in btc_record_df.iterrows():
        transaction_type = row['transaction_type']
        amount = row['other_token_volume']
        price = row['average_price']

        # Calculate PnL for each row based on the difference from the current price
        pnl = (current_price - price) * amount
        
        # For SELL transactions, PnL is negative because it's reducing the position
        if transaction_type == "SELL":
            pnl *= -1

        # Ignore the last row (current price reference)
        if row.name == btc_record_df.index[-1]:
            pnl = 0

        total_pnl += pnl

        # Append the result for this transaction
        results.append({
            "Date": row['dateTime'],
            "timeStamp": row['timeStamp'],
            "Type": transaction_type,
            "amount": amount,
            "price": price,
            "pnl": pnl
        })

    # Convert results to a DataFrame
    pnl_df = pd.DataFrame(results)

    return pnl_df


In [3]:
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2024-12-06 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# print(tabulate(summary, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
unique_tokens_count = record_df_2["token_symbol"].nunique()

unique_tokens = record_df_2["token_symbol"].unique()
results = []

for token in unique_tokens:
    token_df = record_df_2[record_df_2["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
    print(f"{token}: {len(token_df)} records")

    pnl_df = calculate_pnl(token_df)
    print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
    total_pnl = pnl_df["pnl"].sum()

    results.append({"token": token, "total_pnl": total_pnl})

pnl_summary_df = pd.DataFrame(results)

print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

# BTC_record_df = record_df_1[record_df_1["formatted_record"].str.contains(r"\bcbBTC\b", regex=True)]
# # print(BTC_record_df['formatted_record'])
# pnl_df = calculate_pnl(BTC_record_df)
# print(f"Total PnL: {pnl_df['pnl'].sum()}")




2025-01-07 11:28:34,392 - INFO - Start block: 21339732, End block: 21382703
2025-01-07 11:28:34,393 - INFO - Fetching ERC20 transfers from block 21339732 to 21382703...
2025-01-07 11:28:37,118 - INFO - Page 1: Retrieved 119 transactions.
2025-01-07 11:28:38,148 - WARNING - API request failed on page 2: Request failed: No transactions found
2025-01-07 11:28:38,150 - INFO - Final dataset contains 119 unique transactions.
2025-01-07 11:28:38,158 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
2025-01-07 11:28:38,158 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result


Formatted Transactions: part 1
Formatted Transactions: part 2
Formatted Transactions: final part
WLFI: 2 records
+---------------------+-------------+--------+---------------+---------+---------+
| Date                |   timeStamp | Type   |        amount |   price |     pnl |
+=====================+=============+========+===============+=========+=========+
| 2024-10-11 00:57:35 |  1728608255 | 'BUY'  | 2249999999.00 |    0.00 | 1795.30 |
+---------------------+-------------+--------+---------------+---------+---------+
| 2024-11-25 16:13:11 |  1732551191 | 'SELL' | 1000000000.00 |    0.00 |    0.00 |
+---------------------+-------------+--------+---------------+---------+---------+
cbBTC: 38 records
+---------------------+-------------+--------+----------+----------+--------+
| Date                |   timeStamp | Type   |   amount |    price |    pnl |
+=====================+=============+========+==========+==========+========+
| 2024-11-29 17:38:35 |  1732901915 | BUY    |     1.7